# Image recognition algorithm: climate regions of streetview panoramas

In [1]:
import os
from main import make_labelled_dataframe

2023-09-23 18:04:55.780765: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Download the dataset from kaggle. You must use your personal username and key

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets

opendatasets.download(
    "https://www.kaggle.com/datasets/nikitricky/streetview-photospheres"
)

setup the directories and csv path (defaults should be correct)

In [ ]:
main_dir = "streetview-photospheres"
img_dir = os.path.join(main_dir, "images")
csv_path = os.path.join(main_dir, "images.csv")

In [ ]:
# you can choose the dataframe keys for the label and file names
label_key = "label"
file_name_key = "file_name"
df, name_dict = make_labelled_dataframe(
    img_dir, csv_path, label_key=label_key, file_name_key=file_name_key
)